### 令和6年7月6日(土)

#### 不要な特徴量の削除


#### 1. 読み込み

In [ ]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
import lightgbm as lgbm
import xgboost as xgb
from xgboost import XGBClassifier
from hyperopt import hp, fmin, tpe, Trials
import pickle
import os
import sys
sys.path.append('../')
from my_utils import MyUtils
from params import Params
from model.multi_model import Lgbm_hyperopt, Xgb_hyperopt, Cat_hyperopt
import warnings
warnings.filterwarnings('ignore')
INPUT_DIR = "../input/"
RANDOM_STATE = 10
OUTPUT_PATH = "C:/Users/gwsgs/workSpace/GCIcomp2/02.（公開）コンペ2/output/"
remove_outliers_columns = [ "AMT_INCOME_TOTAL", "AMT_ANNUITY", "DAYS_ID_PUBLISH",]
exel_path = 'score_record.xlsx' 
utils = MyUtils(input_path=INPUT_DIR, output_path=OUTPUT_PATH, exel_path=exel_path) 
my_utils = MyUtils(input_path=INPUT_DIR, output_path=OUTPUT_PATH, exel_path=exel_path)
my_params = Params() 

In [ ]:
# データの読み込み
# INPUT_DIRにtrain.csvなどのデータを置いているディレクトリを指定してください。
INPUT_DIR = "../input/"
import datetime
time = datetime.datetime.now().strftime('%Y%m%d%H%M')
train = pd.read_csv(INPUT_DIR + "train.csv")
test = pd.read_csv(INPUT_DIR + "test.csv")
sample_sub = pd.read_csv(INPUT_DIR + "sample_submission.csv")
X_train = train.drop('TARGET',axis=1)
y_train = train['TARGET']

#### 2. 前処理と特徴量作成
ここでは、上記の可視化と分析でわかったことを踏まえて、前処理と特徴量の作成を行います。

In [ ]:
# 欠損値の補完を行う
# FLAF_OWN_CARの欠損値をラベルエンコーディングで補完
train["FLAG_OWN_CAR"].fillna('Unknown', inplace=True)
train["FLAG_OWN_CAR"] = train["FLAG_OWN_CAR"].map(train['FLAG_OWN_CAR'].value_counts().rank(ascending=False, method='first'))
test["FLAG_OWN_CAR"].fillna('Unknown', inplace=True)
test["FLAG_OWN_CAR"] = test["FLAG_OWN_CAR"].map(test['FLAG_OWN_CAR'].value_counts().rank(ascending=False, method='first'))
# FLAG_OWN_REALTYの欠損値はをラベルカウントエンコーディングで補完
train["FLAG_OWN_REALTY"].fillna('Unknown', inplace=True)
train["FLAG_OWN_REALTY"] = train["FLAG_OWN_REALTY"].map(train["FLAG_OWN_REALTY"].value_counts().rank(ascending=False, method='first'))
test["FLAG_OWN_REALTY"].fillna('Unknown', inplace=True)
test["FLAG_OWN_REALTY"] = test["FLAG_OWN_REALTY"].map(test["FLAG_OWN_REALTY"].value_counts().rank(ascending=False, method='first'))
# AMT_ANNUITYの欠損値を中央値で補完
train["AMT_ANNUITY"].fillna(train["AMT_ANNUITY"].median(), inplace=True)
test["AMT_ANNUITY"].fillna(train["AMT_ANNUITY"].median(), inplace=True)
# AMT_GOODS_PRICEの欠損値を中央値で補完
train["AMT_GOODS_PRICE"].fillna(train["AMT_GOODS_PRICE"].median(), inplace=True)
test["AMT_GOODS_PRICE"].fillna(train["AMT_GOODS_PRICE"].median(), inplace=True)
# NAME_TYPE_SUITEの欠損値をラベルカウントエンコーディングで補完
train["NAME_TYPE_SUITE"].fillna("Unknown", inplace=True)
train["NAME_TYPE_SUITE"] = train["NAME_TYPE_SUITE"].map(train["NAME_TYPE_SUITE"].value_counts().rank(ascending=False, method='first'))
test["NAME_TYPE_SUITE"].fillna("Unknown", inplace=True)
test["NAME_TYPE_SUITE"] = test["NAME_TYPE_SUITE"].map(test["NAME_TYPE_SUITE"].value_counts().rank(ascending=False, method='first'))
# OCCUPATION_TYPEの欠損値をラベルエンコーディングで補完
train["OCCUPATION_TYPE"].fillna("Unknown", inplace=True)
train["OCCUPATION_TYPE"] = train["OCCUPATION_TYPE"].map(train["OCCUPATION_TYPE"].value_counts().rank(ascending=False, method='first'))
test["OCCUPATION_TYPE"].fillna("Unknown", inplace=True)
test["OCCUPATION_TYPE"] = test["OCCUPATION_TYPE"].map(test["OCCUPATION_TYPE"].value_counts().rank(ascending=False, method='first'))
# CNT_FAM_MEMBERSの欠損値を中央値で補完
train["CNT_FAM_MEMBERS"].fillna(train["CNT_FAM_MEMBERS"].median(), inplace=True)
test["CNT_FAM_MEMBERS"].fillna(train["CNT_FAM_MEMBERS"].median(), inplace=True)


In [ ]:
# カテゴリカルデータを数値に変換
# NAME_CONTRACT_TYPEのラベルエンコーディング
train["NAME_CONTRACT_TYPE"] = train["NAME_CONTRACT_TYPE"].map({"Cash loans": 0, "Revolving loans": 1})
test["NAME_CONTRACT_TYPE"] = test["NAME_CONTRACT_TYPE"].map({"Cash loans": 0, "Revolving loans": 1})
# CODE_GENDERのラベルエンコーディング(男性=0, 女性=1, XNA=0)
train["CODE_GENDER"] = train["CODE_GENDER"].map({"M": 0, "F": 1, "XNA": 0})
test["CODE_GENDER"] = test["CODE_GENDER"].map({"M": 0, "F": 1, "XNA": 0})
# NAME_INCOME_TYPEのラベルカウントエンコーディング
train["NAME_INCOME_TYPE"] = train["NAME_INCOME_TYPE"].map(train["NAME_INCOME_TYPE"].value_counts().rank(ascending=False, method='first'))
test["NAME_INCOME_TYPE"] = test["NAME_INCOME_TYPE"].map(test["NAME_INCOME_TYPE"].value_counts().rank(ascending=False, method='first'))
# NAME_EDUCATION_TYPEのラベルカウントエンコーディング
train["NAME_EDUCATION_TYPE"] = train["NAME_EDUCATION_TYPE"].map(train["NAME_EDUCATION_TYPE"].value_counts().rank(ascending=False, method='first'))
test["NAME_EDUCATION_TYPE"] = test["NAME_EDUCATION_TYPE"].map(test["NAME_EDUCATION_TYPE"].value_counts().rank(ascending=False, method='first'))
# NAME_FAMILY_STATUSのラベルカウントエンコーディング
train["NAME_FAMILY_STATUS"] = train["NAME_FAMILY_STATUS"].map(train["NAME_FAMILY_STATUS"].value_counts().rank(ascending=False, method='first'))
test["NAME_FAMILY_STATUS"] = test["NAME_FAMILY_STATUS"].map(test["NAME_FAMILY_STATUS"].value_counts().rank(ascending=False, method='first'))
# NAME_HOUSING_TYPEのラベルカウントエンコーディング
train["NAME_HOUSING_TYPE"] = train["NAME_HOUSING_TYPE"].map(train["NAME_HOUSING_TYPE"].value_counts().rank(ascending=False, method='first'))
test["NAME_HOUSING_TYPE"] = test["NAME_HOUSING_TYPE"].map(test["NAME_HOUSING_TYPE"].value_counts().rank(ascending=False, method='first'))
# ORGANIZATIONのラベルカウントエンコーディング
train["ORGANIZATION_TYPE"] = train["ORGANIZATION_TYPE"].map(train["ORGANIZATION_TYPE"].value_counts().rank(ascending=False, method='first'))
test["ORGANIZATION_TYPE"] = test["ORGANIZATION_TYPE"].map(test["ORGANIZATION_TYPE"].value_counts().rank(ascending=False, method='first'))

In [ ]:
# 特徴量の作成
# # DAYS_LAST_PHONE_CHANGEを年に変換
# train["YEARS_PHONR_CHANGE"] = train["DAYS_LAST_PHONE_CHANGE"] / -365
# test["YEARS_PHONR_CHANGE"] = test["DAYS_LAST_PHONE_CHANGE"] / -365
# # DAYS_EMPLOYEDを年に変換
# train["YEARS_EMPLOYED"] = train["DAYS_EMPLOYED"] / 365
# test["YEARS_EMPLOYED"] = test["DAYS_EMPLOYED"] / 365
# # DAYS_REGISTRATIONを年に変換
# train["YEARS_REGISTRATION"] = train["DAYS_REGISTRATION"] / -365
# test["YEARS_REGISTRATION"] = test["DAYS_REGISTRATION"] / -365
# # DAYS_ID_PUBLISHを年に変換
# train["YEARS_ID_PUBLISH"] = train["DAYS_ID_PUBLISH"] / -365
# test["YEARS_ID_PUBLISH"] = test["DAYS_ID_PUBLISH"] / -365
# OCCUPATION_TYPEのLow-skill Laborersを1、それ以外を0に変換。新しく特徴量[is_low_skill]を作成
train["is_low_skill"] = train["OCCUPATION_TYPE"].apply(lambda x: 1 if x == "Low-skill Laborers" else 0)
test["is_low_skill"] = test["OCCUPATION_TYPE"].apply(lambda x: 1 if x == "Low-skill Laborers" else 0)
# 参考 https://www.kaggle.com/competitions/home-credit-default-risk/discussion/64821
# AMT_CREDITとAMT_ANNUITYの比率
train["CREDIT_TO_ANNUITY_RATIO"] = train["AMT_CREDIT"] / train["AMT_ANNUITY"]
test["CREDIT_TO_ANNUITY_RATIO"] = test["AMT_CREDIT"] / test["AMT_ANNUITY"]
# AMT_CREDITとAMT_GOODS_PRICEの比率
train["CREDIT_TO_GOODS_RATIO"] = train["AMT_CREDIT"] / train["AMT_GOODS_PRICE"]
test["CREDIT_TO_GOODS_RATIO"] = test["AMT_CREDIT"] / test["AMT_GOODS_PRICE"]
# AMT_CREDITとAMT_ANNUITYの比率
train["AMT_CREDIT_TO_AMT_ANNUITY"] = train["AMT_CREDIT"] / train["AMT_ANNUITY"]
test["AMT_CREDIT_TO_AMT_ANNUITY"] = test["AMT_CREDIT"] / test["AMT_ANNUITY"]
# DAYS_BIRTH/ -365
train["YEARS_BIRTH"] = train["DAYS_BIRTH"] / -365
test["YEARS_BIRTH"] = test["DAYS_BIRTH"] / -365

In [ ]:
# 不要な特徴量の削除
train.drop(['REGION_RATING_CLIENT', 
            'REG_REGION_NOT_WORK_REGION',
            'REG_CITY_NOT_WORK_CITY',
            'LIVE_REGION_NOT_WORK_REGION'], axis=1, inplace=True)
test.drop(['REGION_RATING_CLIENT',
            'REG_REGION_NOT_WORK_REGION',
            'REG_CITY_NOT_WORK_CITY',
            'LIVE_REGION_NOT_WORK_REGION'], axis=1, inplace=True)

In [ ]:
# 目的変数と説明変数に分割
X = train.drop("TARGET", axis=1).values
y = train["TARGET"].values
X_test = test.values
# 訓練データと評価データに分割
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)
# 訓練データを3分割
x_list = list(np.array_split(X_train, 3))
y_list = list(np.array_split(y_train, 3))

## 3. 予測

In [ ]:
lgb_n_iter = 1000
xgb_n_iter = 10
cat_n_iter = 1000
lr_n_iter =  1

In [ ]:
time = datetime.datetime.now().strftime('%Y%m%d%H%M')
# pickleファイル用のディレクトリを作成
if not os.path.exists(OUTPUT_PATH + time + "_pickle"):
    os.makedirs(OUTPUT_PATH + time + "_pickle", exist_ok=False)

In [ ]:
file_name_list= ['lgb_2_best' , 'lgb_6_best', 'lgb_10']
# pickleファイルの読み込み
model_list = []
for file_name in file_name_list:
    with open(OUTPUT_PATH + time + '_pickle/'+file_name + '.pickle', mode='rb') as f:
        model_list.append(pickle.load(f))

In [ ]:
if 'lgb_2_best' not in model_list:
    lgbm_2_hyper = Lgbm_hyperopt(X_train, y_train, X_valid, y_valid, 
                        params=my_params.lgb, n_iter=lgb_n_iter, random_state=RANDOM_STATE, max_depth=2)
    lgb_2_best = lgbm_2_hyper.hyper_param()
    del lgbm_2_hyper
    print(lgb_2_best)
    my_utils.save_pickle(lgb_2_best, f"{time}_pickle/lgb_2_best")

In [ ]:
lgb_2_best = {
    'bagging_freq': 4,
    'bagging_fraction': 0.939350506933662,
    'feature_fraction': 0.507231264497246,
    'learning_rate': 0.175650897441088,
    'min_child_samples': 20,
    'num_leaves': 6,
    'reg_lamb': 30.6036201910942,
    'n_estimators': 1152,
    'min_data_in_leaf': 29,
}
my_utils.save_pickle(lgb_2_best, f"{time}_pickle/lgb_2_best")

In [28]:
if 'lgb_6_best' not in model_list:
    lgbm_6_hyper = Lgbm_hyperopt(X_train, y_train, X_valid, y_valid, 
                        params=my_params.lgb, n_iter=lgb_n_iter, random_state=RANDOM_STATE, max_depth=6)
    lgb_6_best = lgbm_6_hyper.hyper_param()
    del lgbm_6_hyper
    print(lgb_6_best)
    my_utils.save_pickle(lgb_6_best, f"{OUTPUT_PATH}{time}_pickle/lgb_6_best")

  5%|▍         | 97/2000 [09:12<3:26:18,  6.50s/trial, best loss: -0.7635495413990055]

In [ ]:
if 'lgb_10_best' not in model_list:
    lgbm_10_hyper = Lgbm_hyperopt(X_train, y_train, X_valid, y_valid, 
                        params=my_params.lgb, n_iter=lgb_n_iter, random_state=RANDOM_STATE, max_depth=10)
    lgb_10_best = lgbm_10_hyper.hyper_param()
    del lgbm_10_hyper
    print(lgb_10_best)
    my_utils.save_pickle(lgb_10_best, f"{OUTPUT_PATH}{time}_pickle/lgb_10_best")

In [ ]:
cat_hyper = Cat_hyperopt(X_train, y_train, X_valid, y_valid,
                        params=my_params.cat, n_iter=cat_n_iter, random_state=RANDOM_STATE)
cat_best = cat_hyper.hyper_param()
del cat_hyper
print(cat_best)
my_utils.save_pickle(cat_best, f"{OUTPUT_PATH}{time}_pickle/cat_best")

In [ ]:
xgb_hyper = Xgb_hyperopt(X_train, y_train, X_valid, y_valid,
                    params=my_params.xgb, n_iter=xgb_n_iter, random_state=RANDOM_STATE)
xgb_best = xgb_hyper.hyper_param()
del xgb_hyper
print(xgb_best)
my_utils.save_pickle(xgb_best, f"{OUTPUT_PATH}{time}_pickle/xgb_best")

In [ ]:
print(xgb_best)
my_utils.save_pickle(xgb_best, f"xgb_best_{time}.pkl")

In [ ]:
best_dict = []
best_dict.append(lgb_2_best)
best_dict.append(lgb_6_best)
best_dict.append(lgb_10_best)
best_dict.append(xgb_best)
print("lgb_2_best: ", lgb_2_best)
print("lgb_6_best: ", lgb_6_best)
print("lgb_10_best: ", lgb_10_best)
print("xgb_best: ", xgb_best)
print(y_valid)

In [ ]:
model_list[0]

In [ ]:
# lgb_2_best = model_list[0]
# lgb_6_best = model_list[1]
# lgb_10_best = model_list[2]
# スタッキングhttps://rin-effort.com/2020/02/09/learn-stacking/
# LightGBM max_depth2
lgb_2 = LGBMClassifier(
        learning_rate = lgb_2_best['learning_rate'],
        objective = my_params.lgb['objective'],
        max_depth = 2,
        min_child_samples = lgb_2_best['min_child_samples'],
        n_estimators = lgb_2_best['n_estimators'],
        num_leaves = lgb_2_best['num_leaves'],
        reg_lambda= lgb_2_best['reg_lamb'],
        # sample_sub = lgb_2_best['subsample'],
        # sample_bytree = lgb_2_best['colsample_bytree'],
        bagging_freq = lgb_2_best['bagging_freq'],
        bagging_fraction = lgb_2_best['bagging_fraction'],
        feature_fraction = lgb_2_best['feature_fraction'],
        min_data_in_leaf = lgb_2_best['min_data_in_leaf'],
        verbose = -1,
        random_state = RANDOM_STATE,)

# # LightGBM max_depth6
# lgb_6 = LGBMClassifier(
#         learning_rate = lgb_6_best['learning_rate'],
#         objective = my_params.lgb['objective'],
#         max_depth = 6,
#         # min_child_samples = best['min_child_samples'],
#         n_estimators = lgb_6_best['n_estimators'],
#         num_leaves = lgb_6_best['num_leaves'],
#         reg_lambda= lgb_6_best['reg_lamb'],
#         # sample_sub = lgb_6_best['subsample'],
#         # sample_bytree = lgb_6_best['colsample_bytree'],
#         bagging_freq = lgb_6_best['bagging_freq'],
#         bagging_fraction = lgb_6_best['bagging_fraction'],
#         feature_fraction = lgb_6_best['feature_fraction'],
#         min_data_in_leaf = lgb_6_best['min_data_in_leaf'],
#         verbose = -1,
#         random_state = RANDOM_STATE,)
# # LightGBM max_depth10
# lgb_10 = LGBMClassifier(
#         learning_rate = lgb_10_best['learning_rate'],
#         objective = my_params.lgb['objective'],
#         max_depth = 10,
#         # min_child_samples = best['min_child_samples'],
#         n_estimators = lgb_10_best['n_estimators'],
#         num_leaves = lgb_10_best['num_leaves'],
#         reg_lambda= lgb_10_best['reg_lamb'],
#         bagging_freq = lgb_10_best['bagging_freq'],
#         # sample_sub = lgb_10_best['subsample'],
#         # sample_bytree = lgb_10_best['colsample_bytree'],
#         bagging_fraction = lgb_10_best['bagging_fraction'],
#         feature_fraction = lgb_10_best['feature_fraction'],
#         min_data_in_leaf = lgb_10_best['min_data_in_leaf'],
#         verbose = -1,
#         random_state = RANDOM_STATE,)
# # CatBoost
# cat = CatBoostClassifier(
#         learning_rate = cat_best['learning_rate'],
#         num_boost_round = cat_best['num_boost_round'],
#         depth= cat_best['depth'],
#         verbose = 0,
#         random_state = RANDOM_STATE,)
# # # XGBoost
# # xgb = XGBClassifier(
# #         learning_rate = xgb_best['learning_rate'],
# #         n_estimators = xgb_best['n_estimators'],
# #         max_depth = xgb_best['max_depth'],
# #         subsample = xgb_best['subsample'],
# #         colsample_bytree = xgb_best['colsample_bytree'],
# #         gamma = xgb_best['gamma'],
# #         min_child_weight = xgb_best['min_child_weight'],
# #         verbose = 0,
# #         random_state = RANDOM_STATE,)

# # predicts = [xgb_best, lgb_2_best, lgb_6_best, lgb_10_best]
# predicts = [lgb_2_best, lgb_6_best, lgb_10_best, cat_best]

In [ ]:
# 訓練データと評価データに分割
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)
# 訓練データを3分割
x_list = list(np.array_split(X_train, 3))
y_list = list(np.array_split(y_train, 3))

In [ ]:
# 予測値をターゲットとともにnp.arrayでconcat
pred_list = []
result_list = []
valid_list = []
# model_list = [lgb_2, lgb_6,lgb_10, cat]
model_list = [lgb_2, ]
for i, model in enumerate(model_list):
        print(f"model_start: {i}")
        for k, (x, y) in enumerate(zip(x_list, y_list)):
                model.fit(x, y)
                # validデータを用いて評価
                train_pred = model.predict_proba(x)[:, 1]
                valid_pred = model.predict_proba(X_valid)[:, 1]
                test_pred = model.predict_proba(X_test)[:, 1]
                print(f"{i}_{k}model:train roc_auc: {roc_auc_score(y, train_pred)}")
                print(f"valid roc_auc: {roc_auc_score(y_valid, valid_pred)}")
                vaild_stack = np.stack([valid_pred, y_valid], axis=1)
                # validデータに対する予測値を結合
                if k == 0:
                        vaild_pred_np = vaild_stack
                else:
                        vaild_pred_np = np.concatenate([vaild_pred_np, vaild_stack], axis=0)
                # model毎の予測値をリストに追加
                pred_list.append(test_pred)
        # モデル毎のvalidデータに対する予測値をリストに追加
        valid_list.append(vaild_pred_np)
        # モデル毎の予測値の平均をとる
        test_mean = np.mean(pred_list, axis=0)
        # モデル毎の予測値の平均をリストに追加
        result_list.append(test_mean)
print("finish")

In [ ]:
# pred_npを用いて、スタッキング用のデータセットを作成

x_valid = vaild_pred_np[:,0].reshape(-1,1)
y_valid = vaild_pred_np[:, 1].reshape(-1,1)


In [ ]:
# ロジスティック回帰を用いてスタッキング
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_valid, y_valid)
# 予測
test_preds = np.column_stack(result_list).mean(axis=1).reshape(-1,1)
meta_pred = lr.predict_proba(test_preds)

In [ ]:
meta_pred[:].max()

In [ ]:
dic = {}
for i,  column in enumerate(test.columns):
    dic[i] = column
dic

In [ ]:
# lgbm.plot_importance(lgb,figsize=(10, 10))

#### 4. 予測結果の作成
最後にテストデータに対して予測を行い、提出用のcsvファイルを作成します。

In [ ]:
# テストデータに対する予測値の作成
# pred = lgb.predict_proba(X_test, num_iteration=lgb.best_iteration)[:, 1]
lgb_pred = lgb_2.predict_proba(X_test)[:, 1]
xgb_pred = xgb.predict_proba(X_test)[:, 1]
# lr_pred = lr.predict_proba(X_test_std)[:, 1]                    
# アンサンブル
pred = lgb_pred * 0.5 + xgb_pred * 0.5
# pred = lgb_pred


In [ ]:

# 予測結果を提出用のフォーマットに格納
sample_sub['TARGET'] = meta_pred[:,0].reshape(-1)
# meta_pred[:,1]

In [ ]:
# 提出用のcsvファイルを作成
# formatを指定して現在時刻を取得（yyyyMMddhhmm）

sample_sub.to_csv(f'{OUTPUT_PATH}submit/{time}.csv',index=False)
print(f"output: {OUTPUT_PATH}submit/{time}.csv")